In [10]:
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.layers import Input
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

In [2]:
train_dir = './train'
test_dir = './test'

In [7]:
datagen = ImageDataGenerator(rescale=1./255, 
                             zoom_range=0.02, 
                             rotation_range = 0.02,
                             horizontal_flip=True)
batch_size = 20

In [8]:
train_generator = datagen.flow_from_directory(train_dir,
                                            target_size=(150, 150), # all images will be resized to 150x150
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 750 images belonging to 5 classes.


In [9]:
test_generator = datagen.flow_from_directory(test_dir,
                                            target_size=(150, 150), # all images will be resized to 150x150
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 250 images belonging to 5 classes.


In [13]:
num_classes = 5
input_shape = (150, 150, 3)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras import optimizers
import keras
tb_callback = keras.callbacks.TensorBoard(log_dir='./logs', 
                                          histogram_freq=0, 
                                          batch_size=32, 
                                          write_graph=True, 
                                          write_grads=False, 
                                          write_images=True, 
                                          embeddings_freq=0, 
                                          embeddings_layer_names=None, 
                                          embeddings_metadata=None)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Dropout(0,2))

model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last"))

model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3), padding='same',activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last")) 

model.add(Conv2D(128,(3,3), padding='same',activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last"))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
dropout_11 (Dropout)         (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 75, 75, 64)        18496     
_________________________________________________________________
dropout_12 (Dropout)         (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 75, 75, 64)        36928     
__________

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,
        epochs=20,
        validation_data=test_generator, 
        callbacks=[tb_callback])

Epoch 1/20
38/38 [==============================] - 120s 3s/step - loss: 1.4507 - acc: 0.3842 - val_loss: 1.3718 - val_acc: 0.3800
Epoch 2/20
38/38 [==============================] - 118s 3s/step - loss: 1.2797 - acc: 0.4500 - val_loss: 1.3509 - val_acc: 0.4400
Epoch 3/20
38/38 [==============================] - 122s 3s/step - loss: 1.2326 - acc: 0.4816 - val_loss: 1.1554 - val_acc: 0.4880
Epoch 4/20
38/38 [==============================] - 126s 3s/step - loss: 1.1099 - acc: 0.5237 - val_loss: 1.1131 - val_acc: 0.4920
Epoch 5/20
38/38 [==============================] - 127s 3s/step - loss: 1.0543 - acc: 0.5645 - val_loss: 1.1783 - val_acc: 0.5160
Epoch 6/20
16/38 [===========>..................] - ETA: 1:08 - loss: 1.0797 - acc: 0.5375